In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# Step 1: Review TCGA subdirectories to find the most relevant one for Psoriasis
available_dirs = os.listdir(tcga_root_dir)
print(f"Looking for a relevant directory for {trait} among {len(available_dirs)} TCGA directories")

# Psoriasis is a skin condition. TCGA_Melanoma_(SKCM) is the closest match as it deals with skin cancer
# While not the same disease, it's the closest skin-related dataset in TCGA
relevant_dir = "TCGA_Melanoma_(SKCM)"

# Check if our chosen directory exists
if relevant_dir not in available_dirs:
    print(f"No suitable directory found for {trait}. The closest candidate would be {relevant_dir}.")
    # Record this information and exit
    validate_and_save_cohort_info(is_final=False, cohort="TCGA", info_path=json_path, 
                                 is_gene_available=False, is_trait_available=False)
    exit()
else:
    print(f"Selected {relevant_dir} as the most relevant directory for {trait}")
    
    # Step 2: Identify paths to clinical and genetic data files
    cohort_dir = os.path.join(tcga_root_dir, relevant_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Step 3: Load the clinical and genetic data files
    try:
        clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
        genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
        
        print(f"Clinical data shape: {clinical_df.shape}")
        print(f"Genetic data shape: {genetic_df.shape}")
        
        # Step 4: Print the column names of the clinical data
        print("\nClinical data columns:")
        print(clinical_df.columns.tolist())
        
        # Check if both datasets have data
        is_gene_available = genetic_df.shape[0] > 0 and genetic_df.shape[1] > 0
        is_trait_available = clinical_df.shape[0] > 0 and clinical_df.shape[1] > 0
        
        # Record initial validation
        validate_and_save_cohort_info(is_final=False, cohort="TCGA", info_path=json_path, 
                                     is_gene_available=is_gene_available, is_trait_available=is_trait_available)
        
    except Exception as e:
        print(f"Error loading data: {e}")
        validate_and_save_cohort_info(is_final=False, cohort="TCGA", info_path=json_path, 
                                     is_gene_available=False, is_trait_available=False)


Looking for a relevant directory for Psoriasis among 38 TCGA directories
Selected TCGA_Melanoma_(SKCM) as the most relevant directory for Psoriasis
Clinical data file: TCGA.SKCM.sampleMap_SKCM_clinicalMatrix
Genetic data file: TCGA.SKCM.sampleMap_HiSeqV2_PANCAN.gz


Clinical data shape: (481, 93)
Genetic data shape: (20530, 474)

Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'breslow_depth_value', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_submitted_specimen_dx', 'distant_metastasis_anatomic_site', 'followup_case_report_form_submission_reason', 'form_completion_date', 'gender', 'height', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_weight', 'interferon_90_day_prior_excision_admin_indicator', 'is_ffpe', 'lactate_dehydrogenase_result', 'lost_follow_up', 'mal

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Get clinical data file path
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(f"{tcga_root_dir}/TCGA_Melanoma_(SKCM)")

# Load clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview candidate age columns
if candidate_age_cols:
    age_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_age_cols}
    print("Age columns preview:")
    print(age_preview)

# Extract and preview candidate gender columns
if candidate_gender_cols:
    gender_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_gender_cols}
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [71.0, 82.0, 82.0, 46.0, 74.0], 'days_to_birth': [-26176.0, -30286.0, -30163.0, -17025.0, -27124.0]}
Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# 1. Select appropriate columns for age and gender

# Age column selection
# Both columns seem to contain valid age information
# 'age_at_initial_pathologic_diagnosis' is more directly usable as it's already in years
# 'days_to_birth' would need conversion (it's negative days from birth to diagnosis)
age_col = 'age_at_initial_pathologic_diagnosis'

# Gender column selection
# The 'gender' column looks appropriate with valid values ('MALE', 'FEMALE')
gender_col = 'gender'

# 2. Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features to extract trait (Psoriasis) and demographic info
# For TCGA datasets, we use sample ID patterns to determine the trait (tumor vs normal)
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(f"{tcga_root_dir}/TCGA_Melanoma_(SKCM)")
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
selected_clinical_df = tcga_select_clinical_features(clinical_df, trait, age_col, gender_col)

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Processed clinical data (shape: {selected_clinical_df.shape}) saved to {out_clinical_data_file}")
print(f"Clinical data preview:")
print(selected_clinical_df.head())

# 2. Normalize gene symbols in the genetic data
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
print(f"Original genetic data shape: {genetic_df.shape}")

# Apply normalization using the NCBI Gene database
normalized_genetic_df = normalize_gene_symbols_in_index(genetic_df)
print(f"Normalized genetic data shape: {normalized_genetic_df.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_genetic_df.to_csv(out_gene_data_file)
print(f"Processed gene data saved to {out_gene_data_file}")

# 3. Link clinical and genetic data
# In TCGA datasets, we need to ensure that indexes (sample IDs) match between datasets
common_samples = set(selected_clinical_df.index).intersection(set(normalized_genetic_df.columns))
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

# Filter both dataframes to include only common samples
selected_clinical_df = selected_clinical_df.loc[selected_clinical_df.index.isin(common_samples)]
normalized_genetic_df = normalized_genetic_df[list(common_samples)]

# Combine clinical and genetic data
linked_data = selected_clinical_df.join(normalized_genetic_df.T)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, cleaned_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is the trait distribution biased? {is_biased}")
print(f"Data shape after removing biased features: {cleaned_data.shape}")

# 6. Validate the quality of the data and save metadata
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=cleaned_data,
    note=f"Data from TCGA Melanoma (SKCM) cohort was used as a proxy for {trait}."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Final processed data saved to {out_data_file}")
else:
    print(f"Data was deemed not usable for {trait} analysis - no final file saved.")

Processed clinical data (shape: (481, 3)) saved to ../../output/preprocess/Psoriasis/clinical_data/TCGA.csv
Clinical data preview:
                 Psoriasis   Age  Gender
sampleID                                
TCGA-3N-A9WB-06          1  71.0     1.0
TCGA-3N-A9WC-06          1  82.0     1.0
TCGA-3N-A9WD-06          1  82.0     1.0
TCGA-BF-A1PU-01          1  46.0     0.0
TCGA-BF-A1PV-01          1  74.0     0.0


Original genetic data shape: (20530, 474)
Normalized genetic data shape: (19848, 474)


Processed gene data saved to ../../output/preprocess/Psoriasis/gene_data/TCGA.csv
Number of common samples between clinical and genetic data: 474
Linked data shape: (474, 19851)


Data shape after handling missing values: (474, 19851)
For the feature 'Psoriasis', the least common label is '0' with 1 occurrences. This represents 0.21% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 70.75
Min: 15.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 180 occurrences. This represents 37.97% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Is the trait distribution biased? True
Data shape after removing biased features: (474, 19851)
Data was deemed not usable for Psoriasis analysis - no final file saved.
